In [1]:
import csv
import pickle
import decimal  # 이것은 시간이 오래 걸리기 때문에 주문이 들어기지 않는 경우에만 사용하는 것이 좋다(전송 수량 찾을 때 굳)

# ck
import datetime
# import pytictoc
import sys

sys.setrecursionlimit(1000000)  # 재귀함수 제한 늘리기
import pandas as pd
import numpy as np
import pip
import sys
import requests
import json
# api json 파싱 python 업비트
import jwt  # PyJWT
import uuid
import requests
from urllib.parse import urlencode
import pyupbit  # 업비트#버젼 0.2.8
print(pyupbit.Upbit)
import ccxt  # 바이낸스
print(ccxt)


import pprint
import time
import math

import binance #!pip install binance-connector
import os
import hashlib
from binance.client import Client # python-binance
########선물거래를 위해#######
from binance_f import RequestClient
from binance_f.constant.test import *
from binance_f.base.printobject import *
from binance_f.model.constant import *
from IPython.display import display, clear_output
from binance.spot import Spot #!pip install binance-connector

##############################소수점 정리....##############################################

##@@@ 이 두개도 합치기


###########업비트에서 환율 조회


headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}


def upbit_get_usd_krw():
    try:
        url = 'https://quotation-api-cdn.dunamu.com/v1/forex/recent?codes=FRX.KRWUSD'
        exchange = requests.get(url, headers=headers).json()
        return exchange[0]['basePrice']
    except:
        pass


# https://docs.upbit.com/docs/market-info-trade-price-detail
################################원화 마켄 주문 최소 주문 가격 단위###########################
def UP_Minimum_Order_cash_Size(up_pre):  ##@@여기 전부 가격을 넣는 형식으로 바꿈.
    #     up_pre = pyupbit.get_current_price("KRW-" + symbol)  # 해당코인 가격조회
    # print(up_pre,"원화 최소 주문 가격 단위 조회")
    if (2000000 <= up_pre):
        return 1000
    if (1000000 <= up_pre < 2000000):
        return 500
    if (500000 <= up_pre < 1000000):
        return 100
    if (100000 <= up_pre < 500000):
        return 50
    if (10000 <= up_pre < 100000):
        return 10
    if (1000 <= up_pre < 10000):
        return 5
    if (100 <= up_pre < 1000):
        return 1
    if (10 <= up_pre < 100):
        return 0.1
    if (0 <= up_pre < 10):
        return 0.01


#########################################전송가능 심볼명#########################################
######심볼명 예외처리로 중복제거를 위해######
def removeAllOccur(l, i):
    try:
        while True: l.remove(i)
    except ValueError:
        pass


#########################################
def bin_fu_setting(send_to_bin_optimal_symbol, future_leverage):
    try:
        result = request_client.change_margin_type(symbol=send_to_bin_optimal_symbol + "USDT", marginType=FuturesMarginType.ISOLATED)
    #         PrintBasic.print_obj(result)
    except:#         print("마진타입 변화 불필요")
        pass
    
    try:
        result = request_client.change_initial_leverage(symbol=send_to_bin_optimal_symbol + "USDT", leverage=future_leverage)
    #         PrintBasic.print_obj(result) # 레버리지 변화 (자본에 따라 기본 10배)
    except:
        pass
    
    


#     PrintBasic.print_obj(result)

def up_spot_ask_bid_info(sym_name):
    # 업비트 매수, 매도 호가창 한번에 조회
    up_sym_names = ["KRW-" + sym_name[i] for i in range(len(sym_name))]
    up_spot_ask_bid = pyupbit.get_orderbook(tickers=up_sym_names)
    return up_sym_names, up_spot_ask_bid


def up_ask_info(up_sym_names, up_spot_ask_bid):  # 업비트 매수 호가창
    up_ask_price = []
    up_ask_size = []
    up_ask_price_append = up_ask_price.append
    up_ask_size_append = up_ask_size.append
    for i in range(len(up_sym_names)):
        up_ask_price_append(float(next((item for item in up_spot_ask_bid if item['market'] == up_sym_names[i]), None)['orderbook_units'][0]['ask_price']))
        up_ask_size_append(float(next((item for item in up_spot_ask_bid if item['market'] == up_sym_names[i]), None)['orderbook_units'][0]['ask_size']))
    #         print(up_ask_price[i],up_ask_size[i],up_sym_names[i])
    return up_ask_price, up_ask_size


def up_bid_info(up_sym_names, up_spot_ask_bid):  # 업비트 매도 호가창
    up_bid_price = []
    up_bid_size = []
    up_bid_price_append = up_bid_price.append
    up_bid_size_append = up_bid_size.append
    for i in range(len(up_sym_names)):
        up_bid_price_append(float(next((item for item in up_spot_ask_bid if item['market'] == up_sym_names[i]), None)['orderbook_units'][0]['bid_price']))
        up_bid_size_append(float(next((item for item in up_spot_ask_bid if item['market'] == up_sym_names[i]), None)['orderbook_units'][0]['bid_size']))
    #         print(up_bid_price[i],up_bid_size[i],up_sym_names[i])
    return up_bid_price, up_bid_size

# up_sym_names, up_spot_ask_bid = up_spot_ask_bid_info(sym_name)
# up_ask_price, up_ask_size = up_ask_info(up_sym_names,up_spot_ask_bid)
# up_bid_price, up_bid_size= up_bid_info(up_sym_names,up_spot_ask_bid)
def bin_spot_ask_bid_info(sym_name):  # 바이낸스 현물 호가창 한번에 조회
    bin_sym_names = [sym_name[i] + "USDT" for i in range(len(sym_name))]
    bin_spot_ask_bid = client_spot.book_ticker()
    return bin_sym_names, bin_spot_ask_bid


def bin_ask_info(bin_sym_names, bin_spot_ask_bid):  # 바이낸스 현물 호가창 조회로 매수 호가창 가져옴
    bin_spot_ask_price = []
    bin_spot_ask_size = []
    bin_spot_ask_price_append = bin_spot_ask_price.append
    bin_spot_ask_size_append = bin_spot_ask_size.append
    for i in range(len(bin_sym_names)):
        bin_spot_ask_price_append(float(next((item for item in bin_spot_ask_bid if item['symbol'] == bin_sym_names[i]), None)['askPrice']))
        bin_spot_ask_size_append(float(next((item for item in bin_spot_ask_bid if item['symbol'] == bin_sym_names[i]), None)['askQty']))
    #         print(bin_spot_ask_price[i],bin_spot_ask_size[i],bin_sym_names[i])
    return bin_spot_ask_price, bin_spot_ask_size


def bin_bid_info(bin_sym_names, bin_spot_ask_bid):  # 바이낸스 현물 호가창 조회로 매도 호가창 가져옴
    bin_spot_bid_price = []
    bin_spot_bid_size = []
    bin_spot_bid_price_append = bin_spot_bid_price.append
    bin_spot_bid_size_append = bin_spot_bid_size.append
    
    for i in range(len(bin_sym_names)):
        bin_spot_bid_price_append(float(next((item for item in bin_spot_ask_bid if item['symbol'] == bin_sym_names[i]), None)['bidPrice']))
        bin_spot_bid_size_append(float(next((item for item in bin_spot_ask_bid if item['symbol'] == bin_sym_names[i]), None)['bidQty']))
    #         print(bin_spot_bid_price[i],bin_spot_bid_size[i],bin_sym_names[i])
    return bin_spot_bid_price, bin_spot_bid_size

# bin_sym_names, bin_spot_ask_bid = bin_spot_ask_bid_info(sym_name)
# bin_spot_ask_price,bin_spot_ask_size = bin_ask_info(bin_sym_names,bin_spot_ask_bid)
# bin_spot_bid_price,bin_spot_bid_size = bin_bid_info(bin_sym_names,bin_spot_ask_bid)

def bin_fu_ask_bid_info(sym_name):
    bin_sym_names = [sym_name[i] + "USDT" for i in range(len(sym_name))]
    bin_fu_ask_bid = client.futures_orderbook_ticker()

    return bin_sym_names, bin_fu_ask_bid


def bin_fu_ask_info(bin_sym_names, bin_fu_ask_bid):
    bin_fu_ask_price = []
    bin_fu_ask_size = []
    bin_fu_ask_price_append = bin_fu_ask_price.append
    bin_fu_ask_size_append = bin_fu_ask_size.append
    for i in range(len(bin_sym_names)):
        bin_fu_ask_price_append(float(next((item for item in bin_fu_ask_bid if item['symbol'] == bin_sym_names[i]), None)['askPrice']))
        bin_fu_ask_size_append(float(next((item for item in bin_fu_ask_bid if item['symbol'] == bin_sym_names[i]), None)['askQty']))
    #         print(bin_fu_ask_price[i],bin_fu_ask_size[i],bin_sym_names[i])
    return bin_fu_ask_price, bin_fu_ask_size


def bin_fu_bid_info(bin_sym_names, bin_fu_ask_bid):
    bin_fu_bid_price = []
    bin_fu_bid_size = []
    bin_fu_bid_price_append = bin_fu_bid_price.append
    bin_fu_bid_size_append = bin_fu_bid_size.append
        
    for i in range(len(bin_sym_names)):
        bin_fu_bid_price_append(float(next((item for item in bin_fu_ask_bid if item['symbol'] == bin_sym_names[i]), None)['bidPrice']))
        bin_fu_bid_size_append(float(next((item for item in bin_fu_ask_bid if item['symbol'] == bin_sym_names[i]), None)['bidQty']))
    #         print(bin_fu_bid_price[i],bin_fu_bid_size[i],bin_sym_names[i])
    return bin_fu_bid_price, bin_fu_bid_size

# bin_sym_names,bin_fu_ask_bid = bin_fu_ask_bid_info(sym_name)
# bin_fu_ask_price,bin_fu_ask_size = bin_fu_ask_info(sym_name,bin_fu_ask_bid)
# bin_fu_bid_price,bin_fu_bid_size = bin_fu_bid_info(sym_name,bin_fu_ask_bid)

def find_max_per(second_using_asset, sym_name, bin_spot_ask_price, bin_withdrawfee):
    max_per = []
    max_per_append = max_per.append
    for i in range(len(sym_name)):
        max_per_append(bin_spot_ask_price[i] * bin_withdrawfee[i] / ((second_using_asset / Usdt) / 2))

    return max_per[max_per.index(max(max_per))]


def Adjusting_second_using_asset(second_using_asset, up_market_fee, bin_spot_market_fee, bin_future_market_fee, per, adjust_leverage):
    
    #레버리지를 비율 조정.. 이 부분 맞나?
#     if (adjust_leverage == 1):
#         pass
#     else:
#         adjust_leverage = adjust_leverage - 1
        
    a = (1 - bin_future_market_fee) * adjust_leverage # 바
    b = (1 - bin_spot_market_fee) * (1 - up_market_fee) * (1 - per)  # *(1-0.0004)#@ 0.0004는 추가 보정치 #업

    c = a / (a + b)  # 업
    d = b / (a + b)  # 바

    bin_spot_Usdt_Send_to_up = second_using_asset * c
    bin_fu_Usdt_asset = second_using_asset * d

    return bin_spot_Usdt_Send_to_up, bin_fu_Usdt_asset


# def Adjusting_first_using_asset(sym_name,up_bid_price,up_withdrawfee,up_asset,binance_asset,up_market_fee,bin_spot_market_fee,bin_future_market_fee):
#     max_per = []
#     for i in range(len(sym_name)):
#         max_per.append(up_bid_price[i]* up_withdrawfee[i])
#     max_val= max_per[max_per.index(max(max_per))]

#     a = (1- up_market_fee) * (1-(max_val/up_asset)) *(1- bin_spot_market_fee)
#     b = (1- bin_future_market_fee)

#     c = (a / (a + b)) * (up_asset + binance_asset)
#     d = (b / (a + b)) * (up_asset + binance_asset)
# #     print(c,d)
#     return c,d #바낸자본 업비트 자본

def up_to_bin_canwithdrow_fee(sym_name):
    server_url = 'https://api.upbit.com'
    query = {'currency': sym_name, }
    query_string = urlencode(query).encode()

    m = hashlib.sha512()
    m.update(query_string)
    query_hash = m.hexdigest()

    payload = {'access_key': access_key, 'nonce': str(uuid.uuid4()), 'query_hash': query_hash,
               'query_hash_alg': 'SHA512', }

    jwt_token = jwt.encode(payload, secret_key)
    authorize_token = 'Bearer {}'.format(jwt_token)
    headers = {"Authorization": authorize_token}

    res = requests.get(server_url + "/v1/withdraws/chance", params=query, headers=headers)
    # 해당 코인 전송수수료(해당코인 시가 * 전송 수수료)
    symbol_fee = float(res.json()['currency']['withdraw_fee'])
    #         print(symbol_fee)
    #     upbit_withdraw_fee = float(res.json()['currency']['withdraw_fee']) * up_pre
    #         print(upbit_withdraw_fee,"업비트 보낼 시 심볼 전송 가격")

    bin_sym_send_state = res.json()['withdraw_limit']['can_withdraw']  # 출금지원가능여부

    if (bin_sym_send_state == True):
        pass
    else:
        print(sym_name, "해당코인 업비트에서 바이낸스로 출금 불가상태")

    return symbol_fee, bin_sym_send_state


def up_deposit_address(sym_name):
    server_url = 'https://api.upbit.com'
    ############추가 ... 입출금 상태는 주소로 받아야댐..
    try:
        query = {'currency': sym_name, }
        query_string = urlencode(query).encode()

        m = hashlib.sha512()
        m.update(query_string)
        query_hash = m.hexdigest()

        payload = {'access_key': access_key,
                   'nonce': str(uuid.uuid4()),
                   'query_hash': query_hash,
                   'query_hash_alg': 'SHA512', }

        jwt_token = jwt.encode(payload, secret_key)
        authorize_token = 'Bearer {}'.format(jwt_token)
        headers = {"Authorization": authorize_token}

        res = requests.post(server_url + "/v1/deposits/generate_coin_address", params=query, headers=headers)
        up_add = res.json()['deposit_address']
        bin_sym_send_state_2 = True
        up_seadd = res.json()['secondary_address']

        return up_add, bin_sym_send_state_2, up_seadd
    # print(up_add, up_seadd)

    except KeyError:
        up_add = "해당코인 바이낸스에서 업비트 입금 불가상태"
        up_seadd = "해당코인 바이낸스에서 업비트 입금 불가상태"
        bin_sym_send_state_2 = False
        print(sym_name, "해당코인 바이낸스에서 업비트 입금 불가상태")
        return up_add, bin_sym_send_state_2, up_seadd


def up_deposit_address_2(sym_name):
    #### 업비트 해당코인 입금 주소 생성요청
    try:
        query = {'currency': sym_name, }
        query_string = urlencode(query).encode()

        m = hashlib.sha512()
        m.update(query_string)
        query_hash = m.hexdigest()

        payload = {
            'access_key': access_key,
            'nonce': str(uuid.uuid4()),
            'query_hash': query_hash,
            'query_hash_alg': 'SHA512', }

        jwt_token = jwt.encode(payload, secret_key)
        authorize_token = 'Bearer {}'.format(jwt_token)
        headers = {"Authorization": authorize_token}

        res = requests.post(server_url + "/v1/deposits/generate_coin_address", params=query,
                            headers=headers)
        up_add = res.json()['deposit_address']
        up_sym_send_state = True
        up_seadd = res.json()['secondary_address']
        # print(up_add, up_seadd)
        return up_add, up_sym_send_state, up_seadd

    except KeyError:
        up_add = "해당코인 바이낸스에서 업비트 입금 불가상태"
        up_seadd = "해당코인 바이낸스에서 업비트 입금 불가상태"
        up_sym_send_state = False
        print(sym_name, "해당코인 바이낸스에서 업비트 입금 불가상태")
        return up_add, up_sym_send_state, up_seadd


def up_bin_withdraw_fee(sym_name, all_coin_info):
    # 심볼명,달러 등은 전부 수수료를 저장해두고 나중에 갱신.
    up_withdrawfee = []
    bin_withdrawfee = []
    up_withdrawfee_append = up_withdrawfee.append
    bin_withdrawfee_append = bin_withdrawfee.append
#     all_coin_info = client_spot.coin_info()
    for i in range(len(sym_name)):
        aa = next((item for item in all_coin_info if item['coin'] == sym_name[i]), None)['networkList']
        bin_withdrawfee_append(float(next((item for item in aa if item['network'] == sym_name[i]), None)['withdrawFee']))
        symbol_fee, bin_sym_send_state = up_to_bin_canwithdrow_fee(sym_name[i])
        up_withdrawfee_append(symbol_fee)
        time.sleep(0.4)
    return up_withdrawfee, bin_withdrawfee


def bin_withdraw_fee(all_coin_info, sym_name):
    aa = next((item for item in all_coin_info if item['coin'] == sym_name), None)['networkList']
    bin_withdrawfee = (float(next((item for item in aa if item['network'] == sym_name), None)['withdrawFee']))
    return bin_withdrawfee


def sym_intersection():
    ######바이낸스 달러 선물가능 심볼명#####
    bin_fu_sym = client.futures_symbol_ticker()
    for i in range(len(bin_fu_sym)):
        if (bin_fu_sym[i]['symbol'][-4:] == 'USDT'):
            bin_fu_sym[i] = bin_fu_sym[i]['symbol'][:-4]
        else:
            bin_fu_sym[i] = 'remove'
    removeAllOccur(bin_fu_sym, 'remove')
    ########################################
    #####바이낸스 달러 현물가능 코인 심볼명#USDT만 찾기 #####
    bin_sym = client.get_symbol_ticker()

    for i in range(len(bin_sym)):
        if (bin_sym[i]['symbol'][-4:] == 'USDT'):
            bin_sym[i] = bin_sym[i]['symbol'][:-4]
        else:
            bin_sym[i] = 'remove'
    removeAllOccur(bin_sym, 'remove')
    ########################################################
    ######업비트 코인 심볼명########
    up_sym = pyupbit.get_tickers(fiat="KRW")
    for i in range(len(up_sym)):
        # tickers[i].replace('KRW-','')
        up_sym[i] = up_sym[i][4:]
    #################################
    ######심볼명 교집합#####
    # sym_name = list(set(bin_sym) & set(up_sym))  # @
    sym_name = list(set(bin_fu_sym) & set(bin_sym) & set(up_sym))
    print(len(sym_name), len(sym_name))  # 선물없이 업비트만해선..
    return sym_name


def bin_coin_withdrow_fee(all_coin_info, sym_name):
    try:
        aa = next((item for item in all_coin_info if item['coin'] == sym_name), None)['networkList']
        withdrawfee = float(next((item for item in aa if item['network'] == sym_name), None)['withdrawFee'])
        return withdrawfee
    except:
        withdrawfee = "None_network"
        return withdrawfee
        pass


def bin_coin_withdrow_deopsit_Enable(all_coin_info, sym_name):
    try: #자기자신의 네트워크를 가지고, 현재 바이낸스의 입출금 가능상태를 가져옴
        aa = next((item for item in all_coin_info if item['coin'] == sym_name), None)['networkList']
        bb = next((item for item in aa if item['network'] == sym_name), None)
        depositEnable, withdrawEnable = bb['depositEnable'] , bb['withdrawEnable']
    except:
        depositEnable, withdrawEnable = False ,False

    return depositEnable, withdrawEnable


def del_bin_disable_withdrow_deopsit(all_coin_info, sym_name):
    delet_list = []
    delet_list_append = delet_list.append
    for i in range(len(sym_name)):  # 바이낸스에서 입금 가능 , 전송 불가능
        depositEnable, withdrawEnable = bin_coin_withdrow_deopsit_Enable(all_coin_info, sym_name[i])
        #         print(sym_name[i],depositEnable,withdrawEnable)
        if (depositEnable == False or withdrawEnable == False):
            delet_list_append(sym_name[i])
    name = list(set(sym_name) - set(delet_list))
    return name


def delet_None_network(all_coin_info, sym_name):
    delet_list = []
    delet_list_append = delet_list.append
    for i in range(len(sym_name)):  # 바이낸스에서 전송
        withdrawfee = bin_coin_withdrow_fee(all_coin_info, sym_name[i])
        #         print(sym_name[i],withdrawfee)
        if (withdrawfee == "None_network"):
            delet_list_append(sym_name[i])
    name = list(set(sym_name) - set(delet_list))
    return name



def regester_by_upbit_withdrow_addres(sym_name):
    for i in range(len(sym_name)):
#         print(sym_name[i])
        try:
            bin_address = client_spot.deposit_address(sym_name[i])['address']
            bin_Memo = client_spot.deposit_address(sym_name[i])['tag']
            time.sleep(0.1)
            print(sym_name[i])
            print('bin_address', bin_address)
            print('bin_Memo', bin_Memo)
        except:
            print(sym_name[i],"바이낸스 지갑이 닫혔을 수도 있음")


# def Adjusting_second_using_asset(second_using_asset,up_market_fee,bin_spot_market_fee,bin_future_market_fee):
#     a = (1- bin_future_market_fee) #바
#     b = (1- bin_spot_market_fee) * (1- up_market_fee) #업

#     c = a / (a+b) #업
#     d = b / (a+b) #바

#     bin_spot_Usdt_Send_to_up = second_using_asset * c
#     bin_fu_Usdt_asset = second_using_asset * d
#     return bin_spot_Usdt_Send_to_up , bin_fu_Usdt_asset

def Bin_Minimum_Spot_Order_Size(sym_name):
    Bin_Spot_Minimum_Order_Size = []
    Bin_Spot_Minimum_Order_Size_append = Bin_Spot_Minimum_Order_Size.append
    aa = client.get_exchange_info()

    for i in range(len(sym_name)):
        bb = next((item for item in aa['symbols'] if item['symbol'] == sym_name[i] + 'USDT'), None)
        Bin_Spot_Minimum_Order_Size_append(float(next((item for item in bb['filters'] if item['filterType'] == 'LOT_SIZE'), None)['minQty']))
    #     print(Bin_Spot_Minimum_Order_Size)
    return Bin_Spot_Minimum_Order_Size


# def Bin_Minimum_FU_Order_Size(sym_name):
#     Bin_Fu_Minimum_Order_Size = []
#     Bin_Fu_Minimum_Order_Size_append = Bin_Fu_Minimum_Order_Size.append
#     aa = client.futures_exchange_info()

#     for i in range(len(sym_name)):
#         bb = next((item for item in aa['symbols'] if item['symbol'] == sym_name[i] + 'USDT'), None)
#         Bin_Fu_Minimum_Order_Size_append(float(next((item for item in bb['filters'] if item['filterType'] == 'PERCENT_PRICE'), None)['multiplierDecimal']))
#     #     print(Bin_Fu_Minimum_Order_Size)
#     return Bin_Fu_Minimum_Order_Size

def Bin_Minimum_FU_Order_Size(sym_name):
    Bin_Fu_Minimum_Order_Size = []
    Bin_Fu_Minimum_Order_Size_append = Bin_Fu_Minimum_Order_Size.append
    aa = client.futures_exchange_info()
    
    for i in range(len(sym_name)):
        bb = next((item for item in aa['symbols'] if item['symbol'] == sym_name[i] + 'USDT'), None)
        Bin_Fu_Minimum_Order_Size_append(float(next((item for item in bb['filters'] if item['filterType'] == 'MARKET_LOT_SIZE'), None)['minQty']))
    #     print(Bin_Fu_Minimum_Order_Size)
    return Bin_Fu_Minimum_Order_Size


def Bin_Minimum_Withdraw_Size(symbol,all_coin_info):  # 바이낸스 최소전송가능량
    aa = next((item for item in all_coin_info if item['coin'] == symbol), None)['networkList']
    minWithdrawAmount = float(next((item for item in aa if item['network'] == symbol), None)['withdrawIntegerMultiple'])
    return minWithdrawAmount


def del_bin_disable_deopsit(all_coin_info, sym_name):  # 바이낸스 입금 불가능 코인 삭제
    delet_list = []
    delet_list_append = delet_list.append
    for i in range(len(sym_name)):
        depositEnable, withdrawEnable = bin_coin_withdrow_deopsit_Enable(all_coin_info, sym_name[i])
        #         print(sym_name[i],depositEnable,withdrawEnable)
        if (depositEnable == False):
            delet_list_append(sym_name[i])
    name = list(set(sym_name) - set(delet_list))
    return name


def del_bin_disable_withdraw(all_coin_info, sym_name):  # 바이낸스 출금 불가능 심볼 삭제
    delet_list = []
    delet_list_append = delet_list.append
    for i in range(len(sym_name)):
        depositEnable, withdrawEnable = bin_coin_withdrow_deopsit_Enable(all_coin_info, sym_name[i])
        #         print(sym_name[i],depositEnable,withdrawEnable)
        if (withdrawEnable == False):
            delet_list_append(sym_name[i])
    name = list(set(sym_name) - set(delet_list))
    return name


def del_up_disable_deopsit(sym_name):  # 업비트 입금 불가능 코인 삭제
    server_url = 'https://api.upbit.com'
    delet_list = []
    for i in range(len(sym_name)):

        query = {'currency': sym_name[i], }
        query_string = urlencode(query).encode()

        m = hashlib.sha512()
        m.update(query_string)
        query_hash = m.hexdigest()

        payload = {'access_key': access_key, 'nonce': str(uuid.uuid4()), 'query_hash': query_hash, 'query_hash_alg': 'SHA512', }

        jwt_token = jwt.encode(payload, secret_key)
        authorize_token = 'Bearer {}'.format(jwt_token)
        headers = {"Authorization": authorize_token}

        res = requests.get(server_url + "/v1/withdraws/chance", params=query, headers=headers)
        up_sym_whthdraw_state = res.json()['currency']['wallet_support']

        if 'deposit' in up_sym_whthdraw_state:
            pass
        else:
            delet_list.append(sym_name[i])
            print(sym_name[i], "해당코인 업비트에서 바이낸스로 입금 불가상태")

        time.sleep(0.4)
    name = list(set(sym_name) - set(delet_list))
    return name

def del_up_disable_whthdraw(sym_name):  # 업비트 출금 불가능 코인 삭제
    server_url = 'https://api.upbit.com'
    delet_list = []
    for i in range(len(sym_name)):

        query = {'currency': sym_name[i], }
        query_string = urlencode(query).encode()

        m = hashlib.sha512()
        m.update(query_string)
        query_hash = m.hexdigest()

        payload = {'access_key': access_key, 'nonce': str(uuid.uuid4()), 'query_hash': query_hash, 'query_hash_alg': 'SHA512', }

        jwt_token = jwt.encode(payload, secret_key)
        authorize_token = 'Bearer {}'.format(jwt_token)
        headers = {"Authorization": authorize_token}

        res = requests.get(server_url + "/v1/withdraws/chance", params=query, headers=headers)
        up_sym_whthdraw_state = res.json()['currency']['wallet_support']

        if 'withdraw' in up_sym_whthdraw_state:
            pass
        else:
            delet_list.append(sym_name[i])
            print(sym_name[i], "해당코인 업비트에서 바이낸스로 출금 불가상태")

        time.sleep(0.4)
    name = list(set(sym_name) - set(delet_list))
    return name


#####업비트 => 바이낸스 전송가능정보와 코인 수수료 가져오는코드#########################
def Up_Minimum_Withdraw_Size(sym_name):  # 업비트 최소전송가능량
    server_url = 'https://api.upbit.com'
    query = {'currency': sym_name, }
    query_string = urlencode(query).encode()

    m = hashlib.sha512()
    m.update(query_string)
    query_hash = m.hexdigest()

    payload = {'access_key': access_key, 'nonce': str(uuid.uuid4()), 'query_hash': query_hash,
               'query_hash_alg': 'SHA512', }

    jwt_token = jwt.encode(payload, secret_key)
    authorize_token = 'Bearer {}'.format(jwt_token)
    headers = {"Authorization": authorize_token}

    res = requests.get(server_url + "/v1/withdraws/chance", params=query, headers=headers)
    
    return float(res.json()['withdraw_limit']['fixed']) #이부분 틀려있었음
#     return (0.1)**float(res.json()['withdraw_limit']['fixed'])
    #         print(bin_sym_send_state,sym_name[i],"출금지원가능여부")


def up_bin_withdraw_Size(sym_name, all_coin_info):  # 바이낸스 최소전송가능량
    up_Min_Withdraw_Size = []
    bin_Min_Withdraw_Size = []
    up_Min_Withdraw_Size_append = up_Min_Withdraw_Size.append
    bin_Min_Withdraw_Size_append = bin_Min_Withdraw_Size.append
    for i in range(len(sym_name)):  # 잘되나 확인 중 나중에 지우기
        up_Min_Withdraw_Size_append(Up_Minimum_Withdraw_Size(sym_name[i]))
        bin_Min_Withdraw_Size_append(Bin_Minimum_Withdraw_Size(sym_name[i],all_coin_info))
        time.sleep(0.4)
    return up_Min_Withdraw_Size, bin_Min_Withdraw_Size


def remove_sym_name_1(all_coin_info, sym_name, Usdt, up_asset, per, ban_list, up_market_fee):  # 심볼명 리스트, 달러 환율, 자본대비 최소 퍼센테이지

    up_withdrawfee, bin_withdrawfee = up_bin_withdraw_fee(sym_name, all_coin_info)  # 업비트 전송 수수료 , 바이낸스 전송 수수료
    up_Min_Withdraw_Size, bin_Min_Withdraw_Size = up_bin_withdraw_Size(sym_name, all_coin_info)  # 업비트, 바이낸스 전송최소 가능량
    # Bin_Fu_Minimum_Order_Size = Bin_Minimum_FU_Order_Size(sym_name)  # 바이낸스 선물 최소 주문 가능량
    # Bin_Spot_Minimum_Order_Size = Bin_Minimum_Spot_Order_Size(sym_name)  # 바이낸스 현물 최소 주문가능량
    # 자본대비 수수료 , 잡다한 코인들 제외

    up_sym_names, up_spot_ask_bid = up_spot_ask_bid_info(sym_name)
    up_ask_price, up_ask_size = up_ask_info(up_sym_names, up_spot_ask_bid)  # 업비트 매수 호가창 4
    # up_bid_price, up_bid_size = up_bid_info(up_sym_names, up_spot_ask_bid)  # 업비트 매도 호가창 1

    #     bin_sym_names, bin_spot_ask_bid = bin_spot_ask_bid_info(sym_name)
    #     bin_spot_ask_price, bin_spot_ask_size = bin_ask_info(bin_sym_names, bin_spot_ask_bid)  # 바-현 매수 호가창 2
    #     bin_spot_bid_price, bin_spot_bid_size = bin_bid_info(bin_sym_names, bin_spot_ask_bid)  # 바-현 매도 호가창 3

    # up_to_bin_pre_rate = []
    # bin_to_up_pre_rate = []
    # up_withdrawfee_by_asset = []
    # bin_withdrawfee_by_asset = []

    for i in range(len(sym_name)):

        up_can_order_amout = up_asset * (1 - up_market_fee)  # *0.005이미함
        up_can_order_amout = up_can_order_amout + (up_withdrawfee[i] * up_ask_price[i])  ##완벽한 헷징을 위해 업비트 전송료만큼 추가..#@@이부분..계산
        Decimal_rounding = UP_Minimum_Order_cash_Size(up_ask_price[i])
        up_can_order_amout = (up_can_order_amout // Decimal_rounding) * Decimal_rounding

        can_send_Decimal = up_Min_Withdraw_Size[i]  # 전송가능 최소량@@이게 잇어야 계산 가능
        if (can_send_Decimal == 0):  # 전송 수수료가 0 인경우 나눌 수가 없음
            pass
        else:
            Decimal_rounding, Decimal_rounding_dig = (0.1) ** int(can_send_Decimal + 1), int(can_send_Decimal)  # 소수점 라운딩
            up_can_order_amout = (up_can_order_amout // Decimal_rounding) * Decimal_rounding
            up_can_order_amout = round(up_can_order_amout, Decimal_rounding_dig)

        if (((up_ask_price[i] * up_withdrawfee[i]) / up_can_order_amout) < per):  # 구매가능 정보로 해서 더 깔끔(소수점 절단)
            pass
        else:
            print((up_ask_price[i] * up_withdrawfee[i]) / up_can_order_amout, sym_name[i], "자본대비 수수료가 큰 코인")
            print(up_ask_price[i] * up_withdrawfee[i])
            sym_name[i] = 'remove'

        ###@@@ BNB코인 배제 이 부분 로직으로 대체 가능
        if ((sym_name[i] in ban_list) == True):
            sym_name[i] = 'remove'

    removeAllOccur(sym_name, 'remove')
    print(sym_name, len(sym_name))
    return sym_name


def remove_sym_name_2(all_coin_info, sym_name, Usdt, binance_asset, per, ban_list, bin_spot_market_fee):  # 심볼명 리스트, 달러 환율, 자본대비 최소 퍼센테이지

    up_withdrawfee, bin_withdrawfee = up_bin_withdraw_fee(sym_name, all_coin_info)  # 업비트 전송 수수료 , 바이낸스 전송 수수료
    up_Min_Withdraw_Size, bin_Min_Withdraw_Size = up_bin_withdraw_Size(sym_name, all_coin_info)  # 업비트, 바이낸스 전송최소 가능량
    # Bin_Fu_Minimum_Order_Size = Bin_Minimum_FU_Order_Size(sym_name)  # 바이낸스 선물 최소 주문 가능량
    Bin_Spot_Minimum_Order_Size = Bin_Minimum_Spot_Order_Size(sym_name)  # 바이낸스 현물 최소 주문가능량
    # 자본대비 수수료 , 잡다한 코인들 제외

    # up_sym_names, up_spot_ask_bid = up_spot_ask_bid_info(sym_name)
    # up_ask_price, up_ask_size = up_ask_info(up_sym_names, up_spot_ask_bid)  # 업비트 매수 호가창 4
    # up_bid_price, up_bid_size = up_bid_info(up_sym_names, up_spot_ask_bid)  # 업비트 매도 호가창 1

    bin_sym_names, bin_spot_ask_bid = bin_spot_ask_bid_info(sym_name)
    bin_spot_ask_price, bin_spot_ask_size = bin_ask_info(bin_sym_names, bin_spot_ask_bid)  # 바-현 매수 호가창 2
    # bin_spot_bid_price, bin_spot_bid_size = bin_bid_info(bin_sym_names, bin_spot_ask_bid)  # 바-현 매도 호가창 3

    for i in range(len(sym_name)):

        bin_spot_Usdt_Send = binance_asset - binance_asset * bin_spot_market_fee  # 바이낸스 현물 시장가 수수료
        send_to_up_quantity = (bin_spot_Usdt_Send / bin_spot_ask_price[i])  # 바이낸스 업비트로 보낼 달러 / 티커 가격 = 수량
        send_to_up_quantity = send_to_up_quantity + bin_withdrawfee[i]
        # 전송가능 최소량과 주문가능 최소량으로 잘 자름.
        Decimal_rounding = bin_Min_Withdraw_Size[i]
        if (Decimal_rounding == 0):  # 전송 수수료가 0 인경우 나눌 수가 없음
            pass
        else:
            send_to_up_quantity = (send_to_up_quantity // Decimal_rounding) * Decimal_rounding

        Decimal_rounding = Bin_Spot_Minimum_Order_Size[i]
        bin_spot_to_up_can_order_amount = (send_to_up_quantity // Decimal_rounding) * Decimal_rounding
        bin_spot_to_up_can_order_amount = round(bin_spot_to_up_can_order_amount, 6)
        print(sym_name[i], bin_spot_to_up_can_order_amount, (bin_spot_ask_price[i] * bin_withdrawfee[i]) / (binance_asset), bin_spot_ask_price[i])

        if (((bin_spot_ask_price[i] * bin_withdrawfee[i]) / (binance_asset)) < per):  # 구매가능 정보로 해서 더 깔끔(소수점 절단)
            #             print((up_bid_price[i] * up_withdrawfee[i]) / up_can_order_amout, sym_name[i])
            pass

        else:
            print((bin_spot_ask_price[i] * bin_withdrawfee[i]) / (binance_asset), "자본대비 수수료가 큰 코인")
            print(bin_spot_ask_price[i] * bin_withdrawfee[i])
            sym_name[i] = 'remove'

        ###@@@ BNB코인 배제 이 부분 로직으로 대체 가능
        if ((sym_name[i] in ban_list) == True):
            sym_name[i] = 'remove'

    removeAllOccur(sym_name, 'remove')
    print(sym_name, len(sym_name))
    return sym_name

# 업비트 매수 평단가
def up_avg_buy_price(sym_name):
    payload = {
        'access_key': access_key,
        'nonce': str(uuid.uuid4()), }

    jwt_token = jwt.encode(payload, secret_key)
    authorize_token = 'Bearer {}'.format(jwt_token)
    headers = {"Authorization": authorize_token}

    res = requests.get(server_url + "/v1/accounts", headers=headers)
    aa = float(next((item for item in res.json() if item['currency'] == sym_name), None)['avg_buy_price'])
    bb = float(next((item for item in res.json() if item['currency'] == sym_name), None)["balance"])
    return aa, bb


# def bin_fu_avg_buy_price(sym_name):
#     aa = float(next((item for item in client.futures_account()['positions'] if item['symbol'] == sym_name + 'USDT'), None)['entryPrice'])
#     bb = -(float(next((item for item in client.futures_account()['positions'] if item['symbol'] == sym_name + 'USDT'), None)["positionAmt"]))
#     return aa, bb
# @@ 밑에게 더 낫 지 않나 왜 안대
def bin_fu_avg_buy_price(sym_name, bin_fu_position_info):
    aa = float(next((item for item in bin_fu_position_info['positions'] if item['symbol'] == sym_name + 'USDT'), None)['entryPrice'])
    bb = -(float(next((item for item in bin_fu_position_info['positions'] if item['symbol'] == sym_name + 'USDT'), None)["positionAmt"]))
    return aa, bb


def up_withdrow_request(send_to_bin_optimal_simbol, point_of_Withdrow_UP_amount_1, bin_address, bin_Memo):
    # @ 2.코인 출금 요청#########################
    # point_of_Withdrow_UP_amount_1 조심 가져오든지 해야함
    query = {
        'currency': send_to_bin_optimal_simbol,
        'amount': point_of_Withdrow_UP_amount_1,
        'address': bin_address,
        'secondary_address': bin_Memo,
        'transaction_type': 'default'  # default가 일반출금
    }
    query_string = urlencode(query).encode()

    m = hashlib.sha512()
    m.update(query_string)
    query_hash = m.hexdigest()

    payload = {
        'access_key': access_key,
        'nonce': str(uuid.uuid4()),
        'query_hash': query_hash,
        'query_hash_alg': 'SHA512', }

    jwt_token = jwt.encode(payload, secret_key)
    authorize_token = 'Bearer {}'.format(jwt_token)
    headers = {"Authorization": authorize_token}

    res = requests.post(server_url + "/v1/withdraws/coin", params=query, headers=headers)

    print(res.json())
    send_amount = res.json()['amount']
    send_amount = float(send_amount)
    print(send_amount)


# 업비트 입금주소 요청
def up_deposit_address_request(send_to_up_optimal_simbol):
    query = {'currency': send_to_up_optimal_simbol, }
    query_string = urlencode(query).encode()

    m = hashlib.sha512()
    m.update(query_string)
    query_hash = m.hexdigest()

    payload = {'access_key': access_key,
               'nonce': str(uuid.uuid4()),
               'query_hash': query_hash,
               'query_hash_alg': 'SHA512', }

    jwt_token = jwt.encode(payload, secret_key)
    authorize_token = 'Bearer {}'.format(jwt_token)
    headers = {"Authorization": authorize_token}

    res = requests.post(server_url + "/v1/deposits/generate_coin_address", params=query,
                        headers=headers)
    up_add = res.json()['deposit_address']
    up_seadd = res.json()['secondary_address']
    return up_add, up_seadd

def Up_Minimum_Withdraw_Size_digit(sym_name): #자릿수
    server_url = 'https://api.upbit.com'
    query = {'currency': sym_name, }
    query_string = urlencode(query).encode()

    m = hashlib.sha512()
    m.update(query_string)
    query_hash = m.hexdigest()

    payload = {'access_key': access_key, 'nonce': str(uuid.uuid4()), 'query_hash': query_hash,
               'query_hash_alg': 'SHA512', }

    jwt_token = jwt.encode(payload, secret_key)
    authorize_token = 'Bearer {}'.format(jwt_token)
    headers = {"Authorization": authorize_token}

    res = requests.get(server_url + "/v1/withdraws/chance", params=query, headers=headers)

    return float(res.json()['withdraw_limit']['fixed']) #이부분 틀려있었음

C:\Users\Happy\anaconda3\lib\site-packages\_distutils_hack\__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


<class 'pyupbit.exchange_api.Upbit'>
<module 'ccxt' from 'C:\\Users\\Happy\\anaconda3\\lib\\site-packages\\ccxt\\__init__.py'>


In [2]:
# api 키는 바이낸스와 unbit apikey에 저장해둔다
# api 키 값 로드
with open('apikeys.pickle', 'rb') as fr:
    apikeys = pickle.load(fr)

# 암호키 가져옴.
request_client = RequestClient(api_key=g_api_key, secret_key=g_secret_key)
# request_client_D = RequestClient_D(api_key=g_api_key, secret_key=g_secret_key)
server_url = 'https://api.upbit.com'
Usdt = upbit_get_usd_krw()
# Usdt = 1170
print("Usdt =", Usdt)

# 업비트 잔고 조회 및 해당 코인 가격에 맞는 가격 도출 기능
access_key = apikeys['up_access_key']
secret_key = apikeys['up_secret_key']
upbit = pyupbit.Upbit(access_key, secret_key)

# 바이낸스 잔고조회
binance = ccxt.binance({
    'apiKey': g_api_key,
    'secret': g_secret_key})

binance_fu = ccxt.binance({
    'apiKey': g_api_key,
    'secret': g_secret_key,
    'options': {'defaultType': 'future'}})

client_spot = Spot(key=g_api_key
                   , secret=g_secret_key)

client = Client(api_key=g_api_key,
                api_secret=g_secret_key)

def cheak_about_regester_by_upbit():
    with open('regester_by_upbit.pickle', 'rb') as fr:
        regester_by_upbit = pickle.load(fr)
        past_sym_name = regester_by_upbit["past_sym_name"]

    sym_name = sym_intersection()
    
    New_sym_bols = list(set(sym_name) - set(regester_by_upbit['past_sym_name']))
    
    if(len(set(sym_name) - set(regester_by_upbit['past_sym_name'])) != 0):
        print("==================================warning==================================")
        print("새로운 심볼들 !!!!(업비트 출금주소 등록할것!)",list(set(sym_name) - set(regester_by_upbit['past_sym_name'])))
        print("==================================warning==================================")
        
        print("새로운 출금 주소를 등록하시고 진행하시겠습니까 ? (Y/N)")
        print("Y입력 시 업비트에 출금 주소 입력하고 진행")
        print("N입력 시 새로 추가된 심볼은 사용하지 않고 진행함.")
        input_command  = input()
        
        if (input_command == "Y"):
            import warnings
            warnings.warn("새로운 심볼들 !!!!(업비트 출금주소 등록할것!)", DeprecationWarning)
            sys.exit() ## 강제로 파이썬 종료

        elif(input_command == "N"):
            User_data['ban_list'] = User_data['ban_list'] + New_sym_bols
#             return User_data

        else:
            print("input 입력 오류")
            sys.exit() ## 강제로 파이썬 종료
            
def cheak_about_regester_by_bin():
    with open('regester_by_binance.pickle', 'rb') as fr:
        regester_by_binance = pickle.load(fr)
        past_sym_name = regester_by_binance["past_sym_name"]

    sym_name = sym_intersection()
    
    New_sym_bols = list(set(sym_name) - set(regester_by_binance['past_sym_name']))
    
    if(len(set(sym_name) - set(regester_by_binance['past_sym_name'])) != 0):
        print("==================================warning==================================")
        print("새로운 심볼들 !!!!(업비트 출금주소 등록할것!)",list(set(sym_name) - set(regester_by_binance['past_sym_name'])))
        print("==================================warning==================================")
        
        print("새로운 출금 주소를 등록하시고 진행하시겠습니까 ? (Y/N)")
        print("Y입력 시 업비트에 출금 주소 입력하고 진행")
        print("N입력 시 새로 추가된 심볼은 사용하지 않고 진행함.")
        input_command  = input()
        
        if (input_command == "Y"):
            import warnings
            warnings.warn("새로운 심볼들 !!!!(업비트 출금주소 등록할것!)", DeprecationWarning)
            sys.exit() ## 강제로 파이썬 종료

        elif(input_command == "N"):
            User_data['ban_list'] = User_data['ban_list'] + New_sym_bols
#             return User_data

        else:
            print("input 입력 오류")
            sys.exit() ## 강제로 파이썬 종료
            
def Save_ALL_DATA():
    ## 펑션을 종료시키고 싶을 경우 해당 로직 실행 
    # save data 파이썬 딕셔너리 자료형 데이터 저장.
    with open('Free_Risk_Arbitrage_asset_division.pickle','wb') as fw:
        pickle.dump(asset_division, fw)

    with open('Free_Risk_Arbitrage_User_data.pickle','wb') as fw:
        pickle.dump(User_data, fw)

    for i in range(len(asset_division)):

        print(i ,asset_division[i]['up_asset'] , asset_division[i]['State'])
    print("====================asset_division============================")
    print(asset_division)
    print("=======================User_data==============================")
    print(User_data)
    
def set_premium_asset_division(): # 입력은 %로
    print("%단위로 입력할것 (김프가 그대로)")
    
    print("역프~약김프 최대 수치(프로그램 실행 시 )")
    a_1_1 = float(input())# a_1_1 = 2.5
    print("최소 차익 볼 김프")
    a_2_1 = float(input())# a_2_2 = 1
    print("조정하고 싶은 레버리지 비율 기본 10배")
    adjust_leverage = float(input())
    
    print("역프~약김프 최대 수치(프로그램 실행 시 )",a_1_1,"%")
    print("최소 차익 볼 김프", ((1 + a_2_1 /100) /(1 - a_1_1/100) - 1) * 100,"%")
    
    a_1_1 =  a_1_1 / 100
    a_2_1 =  a_2_1 / 100 
    for i in range(len(asset_division)):
        asset_division[i]['want_premium_step_1_1'] = a_1_1
        asset_division[i]['want_premium_step_2_1'] = a_2_1
        asset_division[i]['adjust_leverage'] = adjust_leverage
        
    return asset_division

def check_logic(asset_division):
    #검산용
    sum_1 = 0
    sum_2 = 0
    for i in range(len(asset_division)):
        sum_1 = sum_1 + asset_division[i]["up_asset"]
        sum_2 = sum_2 + asset_division[i]["binance_asset"] * asset_division[i]['Usdt_first'] * asset_division[i]["future_leverage"]
    print("업비트 = ",sum_1,"바이낸스 = ",sum_2)
    if (sum_1 - 1 < sum_2 < sum_1 + 1 ):# 파이썬이 아주 작은 수를 남겨서
        print("사용중인 자본 맞음")
        
    else:
        import warnings
        warnings.warn("업비트와 바이낸스자본이 맞지 않은 확인 바람.", DeprecationWarning)
        sys.exit() ## 강제로 파이썬 종료    
        
#     if(asset_division[i]["future_leverage"] >= 20 ):
#         import warnings
#         warnings.warn("레버리지가 너무 쎔 해당 컬럼 제거바람.", DeprecationWarning)
#         sys.exit() ## 강제로 파이썬 종료    
        
def up_avg_buy_price_info(): # @@ 나중에 이 부분 다 바꾸기
    payload = {
        'access_key': access_key,
        'nonce': str(uuid.uuid4()), }

    jwt_token = jwt.encode(payload, secret_key)
    authorize_token = 'Bearer {}'.format(jwt_token)
    headers = {"Authorization": authorize_token}

    res = requests.get(server_url + "/v1/accounts", headers=headers)
    return res.json()

Usdt = 1442.5


In [3]:
##@ 0.기본셋팅
Usdt = upbit_get_usd_krw()
print('Usdt', Usdt)

balance = binance.fetch_balance()  # free:보유중인 코인, used:거래진행중인 코인 total:전체코인
print("총 자본", upbit.get_balance("KRW") + float(next((item for item in client.futures_account()['assets'] if item['asset'] == 'USDT'), None)['walletBalance']) * Usdt + balance['USDT']['free'] * Usdt)

# 업비트 현물계좌 원화(현금) 확인
# first_up_asset = 1000000# $$@@ 가정한부분 나중에 바꾸기
# first_up_asset = upbit.get_balance("KRW")
# up_asset = first_up_asset - first_up_asset * 0.0005 # @ up_market_fee # 계좌 현금 확인
# up_asset = up_asset - up_asset * 0.0005 # @ up_market_fee # 수수료 보정치(슬리피지)
# print(up_asset, "업비드 자본(수수료 제외)")
up_asset_cheak = upbit.get_balance("KRW")
print(up_asset_cheak, "업비드 자본")

# 바이낸스 선물계좌 달러 확인
# bin_fu_ass = 350# $$@@가정하고 계산하는 부분 나중에 바꾸기.
fu_asset_cheak =float(next((item for item in client.futures_account()['assets'] if item['asset'] == 'USDT'), None)['availableBalance'])
print(fu_asset_cheak, "바이낸스 선물계좌 달러 잔고 확인:") #bin_fu_ass

# first_bin_fu_ass = bin_fu_ass
# bin_fu_ass = bin_fu_ass - bin_fu_ass * 0.0004 # @ bin_future_market_fee
# binance_asset = bin_fu_ass * Usdt
# print(binance_asset, "원 바이낸스 자본(조정 전)")

#@@@ 벤리스트 마지막에 처리하기. 일단 복잡하니까.=> 새로추가된 심볼명들 !!제하기
# ban_list = ['CHZ', 'ETH', 'ANKR', 'LINK', 'BAT', 'SXP', 'ETC', 'OMG', 'MANA', 'ZRX', 'ENJ', 'STORJ', 'KNC', 'SRM', 'SRM', 'SAND', 'CVC', 'NEO', 'BCH', 'SC', 'QTUM']


Usdt 1442.5
총 자본 1753832.755137545
1585634.29667102 업비드 자본
116.60205093 바이낸스 선물계좌 달러 잔고 확인:


## 바뀌는 펑션들.
2022/10/16 이후

In [4]:
def up_spot_ask_bid_info(sym_name):
    # 업비트 매수, 매도 호가창 한번에 조회
    up_sym_names = ["KRW-" + sym_name[i] for i in range(len(sym_name))]
    up_spot_ask_bid = pyupbit.get_orderbook(ticker=up_sym_names)
    return up_sym_names, up_spot_ask_bid

In [ ]:
수수료 부분 바꿈

# up_market_fee = 0.0005  # 0.05%
# bin_spot_market_fee = binance.fees['trading']['taker']  # maker 0.1% taker 0.1% # 0.001
# bin_future_market_fee = 0.0004  # maker 0.02% taker 0.04%
# bin_Coin_M_future_market_fee = 0.0005  # maker 0.01% taker 0.05% # request_client_D.get_commission_rate("BTCUSD_PERP") 로 알아보면 되는데 안됨

up_market_fee = float(upbit.get_chance("KRW-ETC")["maker_bid_fee"]) # 업비트 코인 현물 시장가 #taker,maker 동일해서 그런지 이것만 존재. @ # 다른 (코인)정보로 받아도 됨
bin_fees = binance_fu.fees
bin_spot_market_fee = bin_fees["trading"]['taker'] # 바이낸스 현물 시장가 수수료 #taker는 시장가   # maker 0.1% taker 0.1% # 0.001
bin_future_market_fee = bin_fees["future"]["trading"]["taker"] # 바이낸스 선물 시장가 수수료 #taker는 시장가 
bin_Coin_M_future_market_fee = bin_fees["delivery"]["trading"]["taker"] # 바이낸스 선물 coin-m 시장가 수수료 #taker는 시장가 

좀 더 고민 후 고칠것

In [108]:
def up_avg_buy_price_info(): # @@ 나중에 이 부분 다 바꾸기 => upbit.get_balances() 로 변환 하면 됨
    payload = {
        'access_key': access_key,
        'nonce': str(uuid.uuid4()), }

    jwt_token = jwt.encode(payload, secret_key)
    authorize_token = 'Bearer {}'.format(jwt_token)
    headers = {"Authorization": authorize_token}

    res = requests.get(server_url + "/v1/accounts", headers=headers)
    return res.json()

In [ ]:
업비트 입출금 가능 정보 더 보기 쉽게 바뀜..
upbit.get_deposit_withdraw_status() 